In [2]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from utils.my_indexing_utils import *
from tqdm import tqdm
from langchain.text_splitter import TokenTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

embeddings = OllamaEmbeddings(
    model="llama3.2",
)

#### Fonction de lecture d'un fichier -> String

#### Consitution des documents .txt -> String -> Document

In [3]:
vector_store = Chroma(
    collection_name="documents_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db", 
)

In [6]:
text_splitter = TokenTextSplitter(chunk_size=10000, chunk_overlap=200)
documents_list = makeDocuments(path="Data/corpus",verbose=True,sample_size=2)
all_splits = []
for doc in documents_list:
    source = doc.metadata["source"]
    print(f"Splitting → {source}")
    
    splits = text_splitter.split_documents([doc])
    
    for i, split in enumerate(splits):
        split.metadata["chunk_id"] = f"{source}_chunk_{i}"
        split.metadata["parent_source"] = source
    
    all_splits.extend(splits)
print(f"Split blog post into {len(all_splits)} sub-documents.")




def save_documents(docs, output_dir="splitted_docs"):
    os.makedirs(output_dir, exist_ok=True)
    
    for i, doc in enumerate(docs):
        filename = f"doc_{doc.metadata['chunk_id']}.txt"
        filepath = os.path.join(output_dir, filename)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(doc.page_content)
save_documents(all_splits)


Split blog post into 0 sub-documents.


In [4]:
batch_size =  1
for i in tqdm(range(0, len(all_splits), batch_size), desc="Vectorizing"):
    batch = all_splits[i:i+batch_size]
    vector_store.add_documents(batch)

Vectorizing: 100%|██████████| 437/437 [31:29<00:00,  4.32s/it]


In [9]:
def retrieve(results):
    context = ""
    for document in results:
        context+= document.page_content
    return context



In [12]:
user_query = "Que se passe t-il a l'assemblée de 1881 à 1882 ?"
results = vector_store.similarity_search(query=user_query)

print(results)
context = retrieve(results)
template = """Question: {question}
Answer with this context : {context}"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.2")

chain = prompt | model

chain.invoke({"question": "What is LangChain?", "context": context})



[Document(id='4fe033b3-d140-41d4-a8d7-cf3e2e18f09e', metadata={'chunk_id': '1905-06-09_chunk_8', 'parent_source': '1905-06-09', 'source': '1905-06-09'}, page_content='ui qui n\'a qu\'une pensée dans une discussion est très redoutable. Vous avez ce genre de force, parce que vous gardez toujours la même préoccupation ; mais vous nous permettrez bien d\'examiner la question avec un peu plus de liberté d\'esprit.\n\nSi vous aviez cette liberté d\'esprit, vous verriez que le cas est déterminé, c\'est le second des cinq cas prévus. Il est bien évident que, s\'il n\'y a plus da catholiques dans une commune, ou s\'il n\'y a plus de protestants - ce qui pourrait encore arriver (On rit)—^l\'église sera retirée à l\'association; on la désaffectera. C\'est très simple.JVous venez à mon aide; je vous en remercie. Vous voterez donc avec moi tout à l\'heure.\n\nM. Eugène Réveillaud. Vous avez répondu à mon objection ; je la faisais très loyalement M. Ribot Vous êtes convaincu ; f espèrequvos voisins 

"Il semble que vous avez une partie d'un débat parlementaire ou d'une discussion politique, mais la partie manquante est inconnue. Cependant, je peux essayer de fournir un contexte et des réponses basées sur les informations disponibles.\n\nLe texte vous montre un débat à la Chambre des députés, où les députés sont chargés de décider d'une proposition du gouvernement. Le vote a eu lieu le 28 mars 1887.\n\nLes deux parties opposées ont voté :\n\n- La majorité absolue : 271 voix\n- La contre-majorité : 21 voix\n\nLe contexte politique de l'époque était marqué par la crise financière et économique, ainsi que des tensions politiques en raison de la réforme électorale. Le gouvernement, dirigé par le Premier ministre Léon Gambetta, a proposé une série de mesures pour résoudre ces problèmes.\n\nLes députés ont voté à l'unanimité pour adopter la proposition du gouvernement, ce qui suggère qu'ils étaient d'accord sur les mesures présentées. Cependant, le fait que 21 voix contre soient très peu 